###Encoder keras snippets###

Old code:

    image_model = Sequential()
    image_model.add(Dense(4096, EMBEDDING_SIZE, init='uniform', activity_regularizer=l2(1e-3)))
    f_image = theano.function([image_model.get_input(train=False)], image_model.get_output(train=False))
    img_output = f_image(image_feats) # CNN input w/ shape (nb_regions, nb_features)
    img_output.shape
    
    print image_model.layers[0].W.get_value().shape, image_model.layers[0].b.get_value().shape
    fig, ax = plt.subplots()
    ax.xaxis.set_visible(False)
    ax.imshow(image_model.layers[0].W.get_value())

Old code:
    
    from networks.keras.birnn import BiDirectionLSTM
    t_b = time.time()
    sent_model = Sequential()
    sent_model.add(BiDirectionLSTM(8856, EMBEDDING_SIZE, activation='relu', output_mode='sum', inner_activation='relu'))
    f_sent = theano.function([sent_model.get_input(train=False)], sent_model.get_output(train=False))
    print 'Compiled BiRNN in {}s'.format(time.time() - t_b)
    
    t_b = time.time()
    sent_output = f_sent(sentence_1h)
    print sent_output.shape
    print 'Sentence embedding in {}: {}'.format(time.time() - t_b, sent_output.shape)

Old code:

    # Compute the word/image scores
    ''' 
    Foreach word s_t:
        max v_i*s_t
    '''
    nb_img_fragments = img_output.shape[0]
    nb_sent_fragments = sent_output.shape[0]
    scores = np.empty((nb_img_fragments, nb_sent_fragments), dtype=np.float32)
    for t in xrange(nb_sent_fragments):
        for i in xrange(nb_img_fragments):
            fragment_out = img_output[i]
            sent = sent_output[t]
            scores[i][t] = np.dot(fragment_out, np.squeeze(sent))
    print scores.shape
    max_scores = np.amax(scores, axis=0)[np.newaxis, :]
    plt.gray()
    fig_feats, ax_feats = plt.subplots(1, 2)
    ax_feats[0].imshow(scores)
    ax_feats[1].imshow(max_scores)
    ax_feats[1].axis('off')
    idx_fragment = np.argmax(scores, axis=0)
    print 'S_kl = {}'.format(np.sum(max_scores))

    # Plot the images with their words
    fig_fragments, ax_fragments = plt.subplots(1, len(idx_fragment))
    fig_fragments.set_size_inches(15, 2)
    for i, idx in enumerate(idx_fragment):
        ax_fragments[i].imshow(image_fragments[idx])
        ax_fragments[i].yaxis.set_visible(False)
        ax_fragments[i].set_title(sentence[i])
        ax_fragments[i].set_xticklabels([])
        ax_fragments[i].set_xlabel(max_scores[0][i])

In [ ]:
%%time

from keras.layers.recurrent import Bidirectional, GRU, LSTM, SimpleRNN
from keras.layers import containers
from keras.layers.core import AutoEncoder, TimeDistributedDense
from keras.layers.embeddings import Embedding

encoder = Sequential()
encoder.add(Bidirectional(GRU, 8856, EMBEDDING_SIZE, activation='relu', inner_activation='relu', return_sequences=True))
encoder.add(TimeDistributedDense(EMBEDDING_SIZE, 8856, activation="softmax"))
encoder.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
sentence = vocab.sentences[vocab.img_to_sent[3][3]]
sentence_1h = np.array(vocab.sentence_to_1h(sentence))
sentence_1h = sentence_1h[np.newaxis, :, :]
#encoder.fit(sentence_1h, sentence_1h, nb_epoch=14, batch_size=16, show_accuracy=True)
encoder.predict(sentence_1h)

In [ ]:
%%time

from keras.layers.recurrent import Bidirectional, GRU, LSTM, SimpleRNN
from keras.layers import containers
from keras.layers.core import AutoEncoder, TimeDistributedDense
from keras.layers.embeddings import Embedding

encoder = Sequential()
encoder.add(Bidirectional(GRU, vocab.token_count, EMBEDDING_SIZE, activation='relu', inner_activation='relu', return_sequences=True))
encoder.add(TimeDistributedDense(EMBEDDING_SIZE, vocab.token_count, activation="softmax"))
encoder.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
ys = map(lambda y: map(vocab.get_1h, y), sent_matrix[:5000])
ys = np.asarray(ys)

In [ ]:
encoder.fit(sent_matrix[:5000], ys)